# GPN Cup 2024. Тематическое моделирование
# Трапер Максим

## Задача
В архиве вы найдете файл, который предварительно оцифровал и обработал Лютик

1. Предложите решение по выделению подмножества тематик и их описания из общего корпуса цитат.
    
2. Предположим, Йеннифер и Трисс поставят эту активность как регулярную, проводя интервью раз в месяц по разным доменам предприятия. Предложите варианты, как Лютику сделать для них легковесную систему по обработке новых цитат из опросов с учетом уже накопленной истории извлеченных тематик и тегов.


Актуальная идея:

1) Найти способ адекватного автоматического исправления ошибок. Валидировать различные методы буду на представленном корпусе цитат. По логике, конечно, можно было бы вручную исправить, потратив 5-10 часов. Вариант конечно рабочий и более качественный, но новые комментарии тоже могут быть шумными. Поэтому нужен автоматическое исправление.

Также чистить данные необходимо, чтобы эмбеддинги текстов были "лучше".

2) Использовать несколько методов формирования тематик. Затем дополнить их результатами работы LLM-модели, которая должна будет сформировать более абстрактные тематики.

3) Загружаем корпус цитата - теги в векторную БД.

4) Далее для любой новой цитаты будут искаться похожие цитаты, браться их теги, как-то аггрегироваться и формироваться результат.

# 0. Теоретическое введение

# 1. Ход решения

## 1.1 Предварительная подготовка

In [ ]:
! git clone https://github.com/MaksimTraper/TopicModel_GPN_Cup_2024.git

fatal: destination path 'TopicModel_GPN_Cup_2024' already exists and is not an empty directory.


### 1.1.1 Установка библиотек

In [ ]:
# RAG
!pip install langchain-community langchain-core

# Исправление опечаток
!pip install pyspellchecker
! git clone https://github.com/ai-forever/sage.git
%cd sage
! pip install .
! pip install -e .[errant]

# LLM
!pip install llama-cpp-python

# словарь
!pip install pymorphy2

fatal: destination path 'sage' already exists and is not an empty directory.
/content/sage
Processing /content/sage
  Preparing metadata (setup.py) ... done
Processing ./wheels/augmentex-1.0.3-py3-none-any.whl (from sage==1.1.0)
augmentex is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
  Created wheel for sage: filename=sage-1.1.0-py3-none-any.whl size=47141 sha256=e039c3e0f640a1359d507f507b33fa90148bcd6df95d5ff9b990550de23a8fbc
  Stored in directory: /tmp/pip-ephem-wheel-cache-85vvn_9q/wheels/f6/fb/b3/1a788356f44a49d04bc8fdc67921c8b7f36cef909ee0b9615e
Successfully built sage
  Attempting uninstall: sage
    Found existing installation: sage 1.1.0
    Uninstalling sage-1.1.0:
      Successfully uninstalled sage-1.1.0
Obtaining file:///content/sage
  Preparing metadata (setup.py) ... done
Processing ./wheels/augmentex-1.0.3-py3-none-any.whl (from sage==1.1.0)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.

In [ ]:
import json
import re
from collections import Counter

from nltk.corpus import stopwords
import nltk
from nltk.util import ngrams
from nltk.metrics import edit_distance
from spellchecker import SpellChecker

from langchain import HuggingFaceHub, LLMChain, PromptTemplate

from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM,LogitsProcessor
from llama_cpp import Llama

import os
import torch
from sage.spelling_correction import T5ModelForSpellingCorruption, RuM2M100ModelForSpellingCorrection, AvailableCorrectors

import pandas as pd

from IPython.display import display, HTML

import difflib

import pymorphy2

nltk.download('stopwords')
# Загрузка стоп-слов для русского языка
stop_words = set(stopwords.words('russian'))

import nltk
from nltk.tokenize import sent_tokenize

# Убедитесь, что nltk скачал необходимые данные
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stop_words = set(stopwords.words('russian'))

In [ ]:
stop_words.remove('не')

In [ ]:
scrolling_css = '''
<style>
    .scrollable-dataframe {
        overflow: auto;
        height: 500px; /* Установите желаемую высоту */
        width: auto;
    }
</style>
'''

### 1.1.2 Загрузка исходных файлов

In [ ]:
%cd ..

/content


In [ ]:
file_path_quotes = '/content/TopicModel_GPN_Cup_2024/data/cintra_phoenix_oils_hr_mgck_feather.json'

with open(file_path_quotes, 'r', encoding='utf-8') as f:
  data = json.load(f)

quotes = []
for quote in data:
  quotes.append(quote['quote'])

In [ ]:
file_path_abbrev = '/content/TopicModel_GPN_Cup_2024/data/abbreviations.txt'
abbrev_text = open(file_path_abbrev).read().split('\n')

abbreviations = {}
for abb in abbrev_text[:len(abbrev_text)-1]:
  abbrev = abb.split(' - ')
  abbreviations[abbrev[0].lower()] = abbrev[1].lower()

In [ ]:
i = 0

for i in range(len(quotes)):
  for key, value in abbreviations.items():
    quotes[i] = quotes[i].replace(key, value)
    quotes[i] = quotes[i].replace(key.upper(), value)

### 1.1.3 Исправление опечаток

Выводы по методам и фреймворкам:
* SpellChecker - работает ужасно, так как не учитывает контекст. А на таком шумном датасете это особенно плохо.
* ai-forever/sage-mt5-large и ai-forever/sage-fredt5-distilled-95m - работают посредственно. Много лишнего меняют, много неправильного не трогают
* ai-forever/RuM2M100-418M и ai-forever/sage-m2m100-1.2B работают лучше всего и более чем неплохо, но обе модели норовят менять специфические термины и всё же плохо работают с шумными цитатами

На данный момент моё решение - использовать ai-forever/RuM2M100-418M (как модель поменьше, показывающая сходные результаты с моделью на 1.2B параметров, 2-3 секунды на цитату. Вопрос, конечно, в каких условиях разрабатываемая модель должна использоваться. Если в онлайне, то время, на самом деле, не лучшее). Дополнительно, проанализировав неправильно заменяемые моделью слов, собрать датасет из таких них и в рамках постпроцессинга заменять их на исходные.

#### 1.1.3.1 Функции

In [ ]:
def remove_punctuation(text):
    #text = re.sub(r'[^\w\s\<"\n\']', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\"', ' ', text)
    text = re.sub(r'\\', ' ', text)
    text = re.sub(r'[\«\»\—\-\💣\*\”\🤑\,\🎉\“]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

def text_to_lower(text):
    return text.lower()

# Заменяем последовательности из более чем двух одинаковых букв на две буквы
# ввиду того, что иногда в словах одна буква повторяется много раз подряд, модель это вероятно путает
def remove_repeated_chars(text):
    return re.sub(r'([^\W\d_])\1{2,}', r'\1\1', text, flags=re.UNICODE)

def preprocess_text(text):
    #text = text_to_lower(text)
    text = remove_punctuation(text)
    text = remove_repeated_chars(text)
    return text

In [ ]:
i = 0

for i in range(len(quotes)):
    quotes[i] = preprocess_text(quotes[i])

In [ ]:
def calculate_count_tokens(text, tokenizer):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

In [ ]:
def split_text(text, tokenizer, max_length=128):
    sentences = sent_tokenize(text)
    parts = []
    current_part = ""

    for sentence in sentences:
        if calculate_count_tokens(current_part, tokenizer) + calculate_count_tokens(sentence, tokenizer) <= max_length:
            current_part += sentence + " "
        else:
            parts.append(current_part.strip())
            current_part = sentence + " "
    if current_part:
        parts.append(current_part.strip())
    return parts

In [ ]:
morph = pymorphy2.MorphAnalyzer()

def get_all_forms(word):
    parses = morph.parse(word)
    return set([f.word for f in parses[0].lexeme])

def is_russian_word(word):
    parses = morph.parse(word)
    return any('NOUN' in p.tag or 'VERB' in p.tag or 'ADJF' in p.tag or 'ADVB' in p.tag or 'INFN' in p.tag for p in parses)

def make_df_with_changes(results, need_output=False):
    data = []

    for original_sentence, corrected_sentence in results.items():
        if need_output:
          print(f"Оригинальное предложение: '{original_sentence}'")
          print(f"Исправленное предложение: '{corrected_sentence}'")
          print('-'*50)

        original_text = original_sentence
        changed_text = corrected_sentence

        # Удаляем знаки препинания
        original_clean = preprocess_text(original_sentence)
        result_clean = preprocess_text(corrected_sentence)

        # Токенизируем очищенные предложения
        original_tokens = original_clean.split()
        result_tokens = result_clean.split()

        # Используем SequenceMatcher для выравнивания токенов
        matcher = difflib.SequenceMatcher(None, original_tokens, result_tokens)

        for tag, i1, i2, j1, j2 in matcher.get_opcodes():
            if tag == 'replace':
                max_len = max(i2 - i1, j2 - j1)
                for idx in range(max_len):
                    o_idx = i1 + idx if i1 + idx < i2 else i2 - 1
                    r_idx = j1 + idx if j1 + idx < j2 else j2 - 1
                    o_word = original_tokens[o_idx]
                    r_word = result_tokens[r_idx]
                    if o_word != r_word:
                        data.append({
                            'original_text': original_text,
                            'changed_text': changed_text,
                            'original_word': o_word.replace('.', ''),
                            'changed_word': r_word.replace('.', '')
                        })
            elif tag == 'delete':
                for o_word in original_tokens[i1:i2]:
                    data.append({
                        'original_text': original_text,
                        'changed_text': changed_text,
                        'original_word': o_word.replace('.', ''),
                        'changed_word': ''
                    })
            elif tag == 'insert':
                for r_word in result_tokens[j1:j2]:
                    data.append({
                        'original_text': original_text,
                        'changed_text': changed_text,
                        'original_word': '',
                        'changed_word': r_word.replace('.', '')
                    })

    df_changed_words = pd.DataFrame(data, columns=['original_text', 'changed_text', 'original_word', 'changed_word'])
    return df_changed_words

def fix_typos(model, tokenizer, sample, device='cpu'):
    tokens = tokenizer(sample, return_tensors='pt')
    output = model.generate(tokens['input_ids'].to(device),
                            attention_mask=tokens['attention_mask'].to(device),
                            do_sample=True, top_k=50, top_p=0.95,
                            num_return_sequences=1)
    result = tokenizer.batch_decode(output.cpu(), skip_special_tokens=True)
    return result[0]

def back_protected_words(sample, sample_results, protected_words):
    print(df_changed_words['original_word'].isin(protected_words))
    # Создаём список изменённых защищённых слов
    protected_changes = df_changed_words[df_changed_words['original_word'].isin(protected_words)]

    # Проходим по сгенерированным предложениям и заменяем изменённые защищённые слова на оригинальные
    corrected_results = []
    for original_text, generated_text in zip(sample, sample_results):
        corrected_text = generated_text
        # Получаем все заменённые защищённые слова в этом предложении
        changes_in_sentence = protected_changes[protected_changes['original_text'] == original_text]
        for idx, row in changes_in_sentence.iterrows():
            original_word = row['original_word']
            changed_word = row['changed_word']
            # Заменяем изменённое слово на оригинальное с учётом границ слов
            pattern = rf'\b{re.escape(changed_word)}\b'
            corrected_text = re.sub(pattern, original_word, corrected_text, flags=re.IGNORECASE)
        corrected_results.append(corrected_text)
    return corrected_results

def generate_fixed_from_samples(model, tokenizer, samples, protected_words, device='cpu'):
    model.eval()
    model = model.to(device)

    sample_results = []
    for sample in samples:
      result = fix_typos(model, tokenizer, sample, device)
      sample_results.append(result)
    # Создаём словарь исходных и исправленных предложений
    results_dict = dict(zip(samples, sample_results))
    # Получаем DataFrame с изменёнными словами
    df_changed_words = make_df_with_changes(results_dict)
    corrected_results = back_protected_words(samples, sample_results, protected_words)

    # Получаем DataFrame с изменёнными словами
    results = dict(zip(samples, corrected_results))
    df_changed_words = make_df_with_changes(results, need_output=True)

    return results, df_changed_words
"""
def generate_fixed_from_samples(model, tokenizer, samples, protected_words, device='cpu'):
    model.eval()
    model.to(device)

    sample_results = []
    for sample in samples:
        parts = split_text(sample, tokenizer, max_length=320)
        generated_parts = []

        for part in parts:
            tokens = tokenizer(part, return_tensors='pt', truncation=True, padding=True, max_length=320)
            input_ids = tokens['input_ids'].to(device)
            attention_mask = tokens['attention_mask'].to(device)

            output = model.generate(
                input_ids,
                attention_mask=attention_mask,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=1.0,
                num_return_sequences=1,
                max_new_tokens=50
            )
            result = tokenizer.decode(output[0], skip_special_tokens=True)
            generated_parts.append(result)

        generated_text = remove_extra_spaces(" ".join(generated_parts))
        sample_results.append(generated_text)

    results_dict = dict(zip(samples, sample_results))
    df_changed_words = make_df_with_changes(results_dict)
    protected_changes = df_changed_words[df_changed_words['original_word'].isin(protected_words)]

    corrected_results = []
    for original_text, generated_text in zip(samples, sample_results):
        corrected_text = generated_text
        changes_in_sentence = protected_changes[protected_changes['original_text'] == original_text]
        for _, row in changes_in_sentence.iterrows():
            original_word = row['original_word']
            changed_word = row['changed_word']
            pattern = rf'\b{re.escape(changed_word)}\b'
            corrected_text = re.sub(pattern, original_word, corrected_text, flags=re.IGNORECASE)
        corrected_results.append(corrected_text)

    results = dict(zip(samples, corrected_results))
    df_changed_words = make_df_with_changes(results)

    return results, df_changed_words
"""

'\ndef generate_fixed_from_samples(model, tokenizer, samples, protected_words, device=\'cpu\'):\n    model.eval()\n    model.to(device)\n\n    sample_results = []\n    for sample in samples:\n        parts = split_text(sample, tokenizer, max_length=320)\n        generated_parts = []\n\n        for part in parts:\n            tokens = tokenizer(part, return_tensors=\'pt\', truncation=True, padding=True, max_length=320)\n            input_ids = tokens[\'input_ids\'].to(device)\n            attention_mask = tokens[\'attention_mask\'].to(device)\n\n            output = model.generate(\n                input_ids,\n                attention_mask=attention_mask,\n                do_sample=True,\n                top_k=50,\n                top_p=0.95,\n                temperature=1.0,\n                num_return_sequences=1,\n                max_new_tokens=50\n            )\n            result = tokenizer.decode(output[0], skip_special_tokens=True)\n            generated_parts.append(result)\n\

#### 1.1.3.2 Инференс

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/RuM2M100-418M")
corrector = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/RuM2M100-418M")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
corrector.model.to(device)

In [ ]:
protected_words = ['плантирe', "Зеленосу",'музiка', 'Зеленосу', 'зеленосу', 'глючит', 'омнибус', 'омнибусе', 'плантир',
                   "гвиндеф", 'мантикорные', 'мантикора', "мантикоры", "мантикору", "алхимиком", "алхимик",
                   "карета", "Скрипториум", "змея", "змеиный", "скрипториумом", "жд", 'алхимическое', 'алхимия',
                   'ЦМФ', 'цмФ']
extended_protected_words = set()
for word in protected_words:
    extended_protected_words.update(get_all_forms(word))

protected_words = list(extended_protected_words) + [s.upper() for s in list(abbreviations.keys())] + (list(abbreviations.keys()))

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
tokenizer = tokenizer
model = corrector

samples = quotes[4:5]
results = dict()

results, df_changed_words = generate_fixed_from_samples(model, tokenizer, samples, protected_words, device=device)

0    False
1    False
2    False
3     True
4    False
5    False
6    False
7    False
8    False
Name: original_word, dtype: bool
                                       original_text  \
3  Теперь всьё так просто! Раньше бумажки летали ...   

                                        changed_text original_word  \
3  Теперь всё так просто!! Раньше бумажки летали,...      плантирe   

  changed_word  
3     плантирe  
Теперь всё так просто!! Раньше бумажки летали, терялись, искать приходилось по всему Зеленостопу. А теперь всё в одном месте на планете... Вся информация под рукой, ничто не пропала... Работать стало гораздо удобнее!!!!!.
Оригинальное предложение: 'Теперь всьё так просто! Раньше бумажки летали терялись искать приходилось по всему Зеленосу. А теперь всё в одном месте на плантирe. Вся инфoрмaция под руkой ничо не пропалaют. Работать стало гораздо удобнее!'
Исправленное предложение: 'Теперь всё так просто!! Раньше бумажки летали, терялись, искать приходилось по всему Зеленосто

In [ ]:
display(HTML(scrolling_css))

# Отображаем DataFrame с указанным классом
display(HTML(df_changed_words.to_html(classes='scrollable-dataframe')))

,original_text,changed_text,original_word,changed_word
0,Теперь всьё так просто! Раньше бумажки летали терялись искать приходилось по всему Зеленосу. А теперь всё в одном месте на плантирe. Вся инфoрмaция под руkой ничо не пропалaют. Работать стало гораздо удобнее!,"Теперь всё так просто!! Раньше бумажки летали, терялись, искать приходилось по всему Зеленостопу. А теперь всё в одном месте на планете... Вся информация под рукой, ничто не пропала... Работать стало гораздо удобнее!!!!!.",всьё,всё
1,Теперь всьё так просто! Раньше бумажки летали терялись искать приходилось по всему Зеленосу. А теперь всё в одном месте на плантирe. Вся инфoрмaция под руkой ничо не пропалaют. Работать стало гораздо удобнее!,"Теперь всё так просто!! Раньше бумажки летали, терялись, искать приходилось по всему Зеленостопу. А теперь всё в одном месте на планете... Вся информация под рукой, ничто не пропала... Работать стало гораздо удобнее!!!!!.",просто!,просто!!
2,Теперь всьё так просто! Раньше бумажки летали терялись искать приходилось по всему Зеленосу. А теперь всё в одном месте на плантирe. Вся инфoрмaция под руkой ничо не пропалaют. Работать стало гораздо удобнее!,"Теперь всё так просто!! Раньше бумажки летали, терялись, искать приходилось по всему Зеленостопу. А теперь всё в одном месте на планете... Вся информация под рукой, ничто не пропала... Работать стало гораздо удобнее!!!!!.",Зеленосу,Зеленостопу
3,Теперь всьё так просто! Раньше бумажки летали терялись искать приходилось по всему Зеленосу. А теперь всё в одном месте на плантирe. Вся инфoрмaция под руkой ничо не пропалaют. Работать стало гораздо удобнее!,"Теперь всё так просто!! Раньше бумажки летали, терялись, искать приходилось по всему Зеленостопу. А теперь всё в одном месте на планете... Вся информация под рукой, ничто не пропала... Работать стало гораздо удобнее!!!!!.",плантирe,планете
4,Теперь всьё так просто! Раньше бумажки летали терялись искать приходилось по всему Зеленосу. А теперь всё в одном месте на плантирe. Вся инфoрмaция под руkой ничо не пропалaют. Работать стало гораздо удобнее!,"Теперь всё так просто!! Раньше бумажки летали, терялись, искать приходилось по всему Зеленостопу. А теперь всё в одном месте на планете... Вся информация под рукой, ничто не пропала... Работать стало гораздо удобнее!!!!!.",инфoрмaция,информация
5,Теперь всьё так просто! Раньше бумажки летали терялись искать приходилось по всему Зеленосу. А теперь всё в одном месте на плантирe. Вся инфoрмaция под руkой ничо не пропалaют. Работать стало гораздо удобнее!,"Теперь всё так просто!! Раньше бумажки летали, терялись, искать приходилось по всему Зеленостопу. А теперь всё в одном месте на планете... Вся информация под рукой, ничто не пропала... Работать стало гораздо удобнее!!!!!.",руkой,рукой
6,Теперь всьё так просто! Раньше бумажки летали терялись искать приходилось по всему Зеленосу. А теперь всё в одном месте на плантирe. Вся инфoрмaция под руkой ничо не пропалaют. Работать стало гораздо удобнее!,"Теперь всё так просто!! Раньше бумажки летали, терялись, искать приходилось по всему Зеленостопу. А теперь всё в одном месте на планете... Вся информация под рукой, ничто не пропала... Работать стало гораздо удобнее!!!!!.",ничо,ничто
7,Теперь всьё так просто! Раньше бумажки летали терялись искать приходилось по всему Зеленосу. А теперь всё в одном месте на плантирe. Вся инфoрмaция под руkой ничо не пропалaют. Работать стало гораздо удобнее!,"Теперь всё так просто!! Раньше бумажки летали, терялись, искать приходилось по всему Зеленостопу. А теперь всё в одном месте на планете... Вся информация под рукой, ничто не пропала... Работать стало гораздо удобнее!!!!!.",пропалaют,пропала
8,Теперь всьё так просто! Раньше бумажки летали терялись искать приходилось по всему Зеленосу. А теперь всё в одном месте на плантирe. Вся инфoрмaция под руkой ничо не пропалaют. Работать стало гораздо удобнее!,"Теперь всё так просто!! Раньше бумажки летали, терялись, искать приходилось по всему Зеленостопу. А теперь всё в одном месте н

In [ ]:
tokenizer = tokenizer
model = corrector

samples = list(results.values())
results = dict()

results, df_changed_words = generate_fixed_from_samples(model, tokenizer, samples, protected_words, device=device)

display(HTML(scrolling_css))

# Отображаем DataFrame с указанным классом
display(HTML(df_changed_words.to_html(classes='scrollable-dataframe')))

KeyboardInterrupt: 

### 1.1.4 Предобработка текстов

In [ ]:
def find_common_bigrams(text_list):
    bigram_counts = Counter()
    for text in text_list:
        tokens = text.split()
        bigrams = ngrams(tokens, 1)
        bigram_counts.update(bigrams)
    return bigram_counts.most_common()

bigram_counts = find_common_bigrams(quotes)
print(bigram_counts)

[(('в',), 828), (('не',), 817), (('и',), 747), (('что',), 677), (('на',), 620), (('как',), 553), (('с',), 434), (('у',), 424), (('так',), 365), (('а',), 360), (('это',), 358), (('я',), 338), (('но',), 304), (('всё',), 267), (('-',), 240), (('мы',), 218), (('В',), 211), (('по',), 210), (('все',), 208), (('просто',), 190), (('вот',), 187), (('нас',), 170), (('бы',), 164), (('сказать,',), 150), (('А',), 135), (('там',), 134), (('уже',), 133), (('было',), 132), (('то',), 132), (('они',), 128), (('когда',), 127), (('для',), 124), (('же',), 118), (('дон',), 115), (('мне',), 113), (('как-то',), 109), (('вот,',), 109), (('ЦМФ',), 108), (('ЗМС',), 107), (('за',), 105), (('ты',), 92), (('бы,',), 92), (('потому',), 91), (('очень',), 89), (('чтобы',), 87), (('меня',), 87), (('эти',), 85), (('потом',), 85), (('если',), 83), (('от',), 83), (('к',), 83), (('только',), 82), (('конечно,',), 79), (('она',), 79), (('например,',), 76), (('из',), 73), (('до',), 71), (('них',), 70), (('тоже',), 69), (('есть

In [ ]:
import string

# Инициализация для русского языка
spell = SpellChecker(language='ru')

def Spell_check_text(text):
  words = text.split()
  corrected_words = [spell.correction(word) if spell.correction(word) is not None else word for word in words]
  return ' '.join(corrected_words)

  return text

def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление повторяющихся символов (например, "оооо" -> "о")
    text = re.sub(r'(.)\1{1,}', r'\1', text)

    # Удаление знаков препинания (если не нужны)
    text = text.translate(str.maketrans('', '', string.punctuation))
    """
    # Удаление стоп-слов
    words = text.split()
    words = [abbreviations[word] if word in abbreviations.keys() else word for word in words]
    text = ' '.join([word for word in words if word not in stop_words])
    """
    return text

preprocessed_quotes = [Spell_check_text(quote) for quote in results]

In [ ]:
results2 = dict(zip(results, preprocessed_quotes))

In [ ]:
df_changed_words = make_df_with_changes(results2)

Оригинальное предложение: '«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной  таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне было уже пару раз, чтоо сами по себе грохаются эти полки с буттылками» 


'
Исправленное предложение: 'у нас среди ночь я районе 55 часов упала пола я водой я сттеклянной карен тоесть посреди этой луна сстеккла, всё это уже под утром кстати это не первый раз,,, они сами по себе падают какого неправильно рассчитываю мы же должны заставлять по определённо картинка у нас при мне было уже пару раз что сами по себе грохаются эти полки я буттылками»'
--------------------------------------------------
Оригинальное предложение: 'Программы повышения квалификации через гильдию ЦМФ, хитрые, хитрые.  У всех, кто хочет повышаться, есть свои предпочтения и направления, а гильдия... г

In [ ]:
# Применяем CSS-стиль
display(HTML(scrolling_css))

# Отображаем DataFrame с указанным классом
display(HTML(df_changed_words.to_html(classes='scrollable-dataframe')))

,original_text,changed_text,original_word,changed_word
0,"«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне было уже пару раз, чтоо сами по себе грохаются эти полки с буттылками» \n\n\n","у нас среди ночь я районе 55 часов упала пола я водой я сттеклянной карен тоесть посреди этой луна сстеккла, всё это уже под утром кстати это не первый раз,,, они сами по себе падают какого неправильно рассчитываю мы же должны заставлять по определённо картинка у нас при мне было уже пару раз что сами по себе грохаются эти полки я буттылками»",«у,у
1,"«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне было уже пару раз, чтоо сами по себе грохаются эти полки с буттылками» \n\n\n","у нас среди ночь я районе 55 часов упала пола я водой я сттеклянной карен тоесть посреди этой луна сстеккла, всё это уже под утром кстати это не первый раз,,, они сами по себе падают какого неправильно рассчитываю мы же должны заставлять по определённо картинка у нас при мне было уже пару раз что сами по себе грохаются эти полки я буттылками»",ночi,ночь
2,"«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне было уже пару раз, чтоо сами по себе грохаются эти полки с буттылками» \n\n\n","у нас среди ночь я районе 55 часов упала пола я водой я сттеклянной карен тоесть посреди этой луна сстеккла, всё это уже под утром кстати это не первый раз,,, они сами по себе падают какого неправильно рассчитываю мы же должны заставлять по определённо картинка у нас при мне было уже пару раз что сами по себе грохаются эти полки я буттылками»",в,я
3,"«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне было уже пару раз, чтоо сами по себе грохаются эти полки с буттылками» \n\n\n","у нас среди ночь я районе 55 часов упала пола я водой я сттеклянной карен тоесть посреди этой луна сстеккла, всё это уже под утром кстати это не первый раз,,, они сами по себе падают какого неправильно рассчитываю мы же должны заставлять по определённо картинка у нас при мне было уже пару раз что сами по себе грохаются эти полки я буттылками»",полxа,пола
4,"«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне было уже пару раз, чтоо сами по себе грохаются эти полки с буттылками» \n\n\n","у нас среди ночь я районе 55 часов упала пола я водой я сттеклянной карен тоесть посреди этой луна сстеккла, всё это уже под утром кстати это не первый раз,,, они сами по себе падают какого неправильно рассчитываю мы же должны заставлять по определённо картинка у нас при мне было уже пару раз что сами по себе грохаются эти полки я буттылками»",с,я
5,"«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне

In [ ]:
df_changed_words['original_word'].value_counts().head(15)

NameError: name 'df_changed_words' is not defined

In [ ]:
# Шаг 1: Собираем уникальные слова из всех текстов
unique_words = set(word for quote in preprocessed_quotes[:1] for word in quote.split())

# Шаг 2: Формируем словарь предполагаемых исправлений
correction_dict = {}
for word in unique_words:
    if word not in spell:  # Если слово считается ошибочным
        correction_dict[word] = spell.correction(word)

# Шаг 3: Печатаем словарь для ручной проверки и корректировки
print("Словарь предложенных исправлений:")
for original, correction in correction_dict.items():
    print(f"{original}: {correction}")

"""
# Ручная корректировка (пример)
correction_dict["класный"] = "классный"
correction_dict["Привеетт"] = "Привет"

# Шаг 4: Применение исправлений к тексту
def correct_text_with_dict(text, correction_dict):
    words = text.split()
    corrected_words = [correction_dict.get(word, word) for word in words]
    return ' '.join(corrected_words)

# Пример использования
text = "Привеетт всем, это класный день"
corrected_text = correct_text_with_dict(text, correction_dict)
print("Исправленный текст:", corrected_text)
"""

Словарь предложенных исправлений:
грохаются: None
полxа: пола
таре: тара
«у: у
стекляной: None
както: като
лужа: луна
5: 5
ночi: ночь
бутылками»: None
картинке: картинка
выставлять: заставлять
первыи: первый
расчитывают: рассчитываю
определёной: определённо


'\n# Ручная корректировка (пример)\ncorrection_dict["класный"] = "классный"\ncorrection_dict["Привеетт"] = "Привет"\n\n# Шаг 4: Применение исправлений к тексту\ndef correct_text_with_dict(text, correction_dict):\n    words = text.split()\n    corrected_words = [correction_dict.get(word, word) for word in words]\n    return \' \'.join(corrected_words)\n\n# Пример использования\ntext = "Привеетт всем, это класный день"\ncorrected_text = correct_text_with_dict(text, correction_dict)\nprint("Исправленный текст:", corrected_text)\n'

In [ ]:
edit_distance('цмф', 'мы')

2

In [ ]:
word='Привеетт'

correction = spell.correction(word)

if spell.known([correction]) and edit_distance(word, correction) < 2:
    print(correction)
else:
    print(word)

привет


In [ ]:
# Инициализация для русского языка
spell = SpellChecker(language='ru')

def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление повторяющихся символов (например, "оооо" -> "о")
    text = re.sub(r'(.)\1{1,}', r'\1', text)

    # Удаление знаков препинания (если не нужны)
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Удаление стоп-слов
    words = text.split()
    text = ' '.join([word for word in words if word not in stop_words])

    words = text.split()
    corrected_words = [spell.correction(word) if word not in spell else word for word in words]
    return ' '.join(corrected_words)

    return text

# Пример текста
text = "Привеетт!! Как дела???😊😊😊 Как твоя раб0тта в ЦМФ?"

# Предобработка текста
preprocessed_text = preprocess_text(text)
print("Предобработанный текст:", preprocessed_text)


Предобработанный текст: привет делай твоя работа мы


## 2.1 Основная часть

## 2.1.1 Генерация тегов

In [ ]:
llm = Llama.from_pretrained(
	repo_id="RefalMachine/ruadapt_qwen2.5_3B_ext_u48_instruct_v4_gguf",
	filename="FP16.gguf",
  use_gpu=True,
  device_map="auto",
  n_ctx=2048
)

In [ ]:
import ctypes

# Получение указателя на функцию mlock
libc = ctypes.CDLL("libc.so.6")  # Для Linux; на macOS используйте "libSystem.dylib"
mlock = libc.mlock

# Блокировка памяти для предотвращения выгрузки
def lock_memory(address, length):
    result = mlock(address, length)
    if result != 0:
        raise RuntimeError("Ошибка при вызове mlock для блокировки памяти")

# Получение размера памяти модели (например, 1 ГБ)
model_memory_size = 1 * 1024 * 1024 * 1024 * 12  # 1 ГБ
address = ctypes.c_void_p(id(llm))  # Получаем адрес объекта модели

lock_memory(address, model_memory_size)


In [ ]:
unique_tags = []
for text in quotes[0:10]:
  print(text)
  prompt = (
      f"Далее будет дан текст. Для него нужно сгенерировать от 5 до 10 \
      тегов (длиной от одного до трёх слов), точно описывающих то, что \
      происходит в тексте и о чём или о ком говориться в нём. Если в тексте есть имена собственные или аббревиатуры, обязательно добавляй их в теги.\
      Пример, дан текст: Мурр... Новиград - город контрастов, да. Но в ЦМФ, как и в других ЗМС, люди превратились в котов, все заняты только собой. Никто не хочет протянуть лапку товарищу, помочь с грузом. Все в напряжении, в спешке. Мурр, раньше было лучше, когда работали втроем, а не порознь.\
      Теги: Новиград, ЦМФ, ЗМС, эгоизм, раньше было лучше, Работа"
      f"Текст: {text}; \n Теги:"
  )

  tags = generate_tags(prompt, llm)
  unique_tags.extend(tags)
  print("Generated tags:", tags)

# Удаление дубликатов
unique_tags = list(set(unique_tags))

«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной  таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне было уже пару раз, чтоо сами по себе грохаются эти полки с буттылками» 





Llama.generate: 3 prefix-match hit, remaining 311 prompt tokens to eval
llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   311 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    25 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   57022.62 ms /   336 tokens
Llama.generate: 186 prefix-match hit, remaining 88 prompt tokens to eval


Generated tags: ['Уличные инциденты', 'полxа с водой', 'водно-опасные предметы', 'расчётные ошибки', 'безопасность в районе']
Программы повышения квалификации через гильдию ЦМФ, хитрые, хитрые.  У всех, кто хочет повышаться, есть свои предпочтения и направления, а гильдия... гильдия толкает свое, не всегда то, что нужно сотрудникам.  В "Белых Змеях", например,  программы гибче,  и сотрудники сами выбирают, что им нужно. 



llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    88 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    15 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   20571.85 ms /   103 tokens
Llama.generate: 186 prefix-match hit, remaining 93 prompt tokens to eval


Generated tags: ['ЦМФ', 'гильдия', 'повышения квалификации', 'предпочтения', 'гибкие программы']
"Мурр... Новиград - город контрастов, да. Но в ЦМФ, как и в других ЗМС, люди превратились в котов, все заняты только собой. Никто не хочет протянуть лапку товарищу, помочь с грузом. Все в напряжении, в спешке. Мурр, раньше было лучше, когда работали втроем, а не порознь. " 



...

Generated tags: ['технолог', 'станция', 'зарплаты', 'мечты', 'Крыса']
Вот, знаете, раньше приходилось порой подолгу ждать, пока заправишся. А сейчас, с этoй предоплатой, всё так гладко!  Заезжаешь, оплачиваешь, и вперёд!  Никаких очередей, никаких задержек. Просто супер! 



llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    68 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    19 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   19524.02 ms /    87 tokens
Llama.generate: 186 prefix-match hit, remaining 105 prompt tokens to eval


Generated tags: ['Теги: ожидание', 'заправка', 'предоплата', 'очередь', 'задержка', 'комфорт']
Ну, вот, в Карбоне, например, всё норaльно. Вода чистая, прямо из крана. А вот в глубинке, где ЗМС открываються в новыx меcтах, бывает, приходится жdacь доставки.  Иногда, прав4а, бывает, что... э-э... что вода не такая чиcтая, как хоч4лось бы. Но, в обшему, всё терпимо. 



llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   105 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    19 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   25538.38 ms /   124 tokens


Generated tags: ['Карбоновая вода', 'ЗМС', 'добыча', 'неочевидные трудности', 'терпимость']


In [ ]:
for text in quotes[12:15]:
  print(text)
  prompt = (
          f"Есть ряд тегов, перечисленных через запятую: {', '.join(unique_tags)}"
          "Далее будет дан текст. Выбери 5-7 из данного списка тегов только те, которые точно характеризуют данный текст. Смотри на ключевые слова или их синонимы. Но таких тегов может и не быть\
          Теги: Новиград, ЦМФ, ЗМС, эгоизм, раньше было лучше, Работа"
          f"Текст: {text}\n Теги:"
      )

  tags = generate_tags(prompt, llm)
  print("Generated tags:", tags)

«У нас каждая смена, как маленькая семья. Естественно, все взрослые люди, все с разным темпераментом и характером, но чтобы они какие-то были критические, которые нужно было решать, вплоть до перехода из этой смены в другую, потому что не нашли общего языка, нет, у меня таких вопросов не было»


Llama.generate: 236 prefix-match hit, remaining 98 prompt tokens to eval
llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    98 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    31 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   31999.95 ms /   129 tokens
Llama.generate: 261 prefix-match hit, remaining 68 prompt tokens to eval


Generated tags: ['ЗМС', 'эгоизм', 'раньше было лучше', 'РаботаТекст', 'Зренья', 'ЦМФ', 'Развитие', 'Крыса', 'Мурр.']
Хотчу скать, шо работа в ЦМФ - ето просто супеRRR! Зарплата, конечно, выше чем в "Скиррисе", а скидки на масле в наших маслостанциях просто бомба!  💣 123  



llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    68 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    33 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   28880.37 ms /   101 tokens
Llama.generate: 261 prefix-match hit, remaining 125 prompt tokens to eval


Generated tags: ['Теги: ЦМФ', 'Зарплата', 'Скидки', 'Маслостанция', 'РаботаТекст', 'Эгоизм', 'Раньше было лучше']
В ЦМФ, так ска3ть, зарплата вроде и не малaя, но вот возможности для роста, так ска3ть, ограничены. Я, так ска3ть, ужe полгода как технолог, но всё эшe на том же уровне, так ска3ть, как и пришел. В "Коимбрах Масел", так ска3ть, говорЯт, что продвижение по карьерной лестнице там, так ска3ть, более динамичное. Вот и думaю, так ска3ть, стоит ли менять работу. 





llama_perf_context_print:        load time =   41866.94 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   125 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    18 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   27384.63 ms /   143 tokens


Generated tags: ['ЦМФ', 'ЗМС', 'эгоизм', 'раньше было лучше', 'повышение квалификации', 'развитие', 'сотрудники']
